<a href="https://colab.research.google.com/github/ikcod/SQL_Coder/blob/main/SQLCoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SQLCoder-7b-2**
# Defog LLM Model
Text to SQL LLM

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.cuda.is_available()

True

In [ ]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [ ]:
print(available_memory)

15835660288


# Download the Model


In [ ]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 16e9:
  # if you have atleast 15GB of GPU memory, run the model in float16
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      torch_dtype=torch.float16,
      device_map="auto",
      use_cache = True
      )
else:
  # load in 8 bits - this is slower
  model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code = True,
        #torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache = True
        )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# **Set the Question & Prompt and Tokenize**

In [ ]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the questions with available schema, return 'I dont know'

### Database Schema
CREATE TABLE IF NOT EXISTS passengers(
    "id" INTEGER,
    "first_name" TEXT NOT NULL,
    "last_name" TEXT NOT NULL,
    "age" INTEGER NOT NULL,
    PRIMARY KEY ("id")
);

CREATE TABLE IF NOT EXISTS airlines(
    "id" INTEGER,
    "name" TEXT NOT NULL,
    "concourse" TEXT NOT NULL CHECK ("concourse" IN ('A','B','C','D','E','F','T')),
    PRIMARY KEY ("id")
);

CREATE TABLE IF NOT EXISTS flights(
    "id" INTEGER,
    "flight_no" INTEGER,
    "airline_id" INTEGER,
    "dep_airport" TEXT NOT NULL,
    "arr_airport" TEXT NOT NULL,
    "dep_datetime" NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
    "arr_datetime" NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY ("id"),
    FOREIGN KEY ("airline_id") REFERENCES airlines("id")
);

CREATE TABLE IF NOT EXISTS check_ins(
    "id" INTEGER,
    "flight_id" INTEGER,
    "datetime" NUMERIC NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY ("id"),
    FOREIGN KEY ("flight_id") REFERENCES flights("id")
);

-flights.airline_id can be joined with airlines.id
-flights.id can be joined with check_ins.flight_id

###Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

# **Generate the SQL**

In [ ]:
import sqlparse

def generate_query(question):
  updated_prompt = prompt.format(question=question)
  inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
  generated_ids = model.generate(
      **inputs,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.eos_token_id,
      max_new_tokens=400,
      do_sample=False,
      num_beams=1,
  )
  outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

  torch.cuda.empty_cache()
  torch.cuda.synchronize()
  #empty the cache so that you generate more results w/o memory crash

  # parse the generated SQL
  parsed_sql = sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)
  return parsed_sql

In [ ]:
question = "How many passengers are there?"
sql = generate_query(question)
print(sql)


SELECT 'I don't know' AS answer;


In [ ]:
question = "When does a flight check-ins at the airport?"
generated_sql = generate_query(question)
print(generated_sql)


SELECT c.flight_id,
       to_char(c.datetime, 'YYYY-MM-DD HH24:MI:SS') AS check_in_time
FROM check_ins c;
